This example performs a depletion/transmutation/activation simulation

The simulation has been accelerated by making use of the IndependentOperator instead of the CoupledOperator.

This is an approximation so is less accurate but it is much faster.

This approach performs just a single transport simulation and obtains reactions rates once and assumes that they remain constant.

If the materials don't change significantly during the irradiation this is a reasonable approximation.

Fission fuel pins would perhaps require the full CoupledOperator while the majority of fusion simulations are suitable for the IndependentOperator

More details on both Operators in the docs
https://docs.openmc.org/en/stable/usersguide/depletion.html#transport-independent-depletion

In [ ]:
# remove any old files
!rm settings.xm model.xml materials.xml geometry.xml settings.xml

import openmc
import openmc.deplete
import math
import matplotlib.pyplot as plt

In [ ]:
# chain and cross section paths have been set on the docker image but you may want to change them
#openmc.config['chain_file']=path to chain file
#openmc.config['cross_sections']=path to cross_sections.xml

Creates a simple material which we will deplete

In [ ]:
my_material = openmc.Material(material_id=1) 
my_material.add_element('Ag', 1, percent_type='ao')
my_material.set_density('g/cm3', 10.49)

As we are doing a depletion simulation we must set the material volume and the .depletion to True

In [ ]:
sphere_radius = 100
volume_of_sphere = (4/3) * math.pi * math.pow(sphere_radius, 3)
my_material.volume = volume_of_sphere  # a volume is needed so openmc can find the number of atoms in the cell/material
my_material.depletable = True  # depletable = True is needed to tell openmc to update the material with each time step
materials = openmc.Materials([my_material])

makes a simple sphere surface and cell

In [ ]:
sph1 = openmc.Sphere(r=sphere_radius, boundary_type='vacuum')
shield_cell = openmc.Cell(region=-sph1)
shield_cell.fill = my_material
geometry = openmc.Geometry([shield_cell])

creates a simple point source

In [ ]:
source = openmc.IndependentSource()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
source.particles = 'neutron'

defines the simulation settings

In [ ]:
settings = openmc.Settings()
settings.batches = 10
settings.inactive = 0
settings.particles = 1000
settings.source = source
settings.run_mode = 'fixed source'

builds the model combining the materials, geometry and settings into one object

In [ ]:
model = openmc.model.Model(geometry, materials, settings)

this does perform particle transport but just to get the flux and micro xs

In [ ]:
flux_in_each_group, micro_xs = openmc.deplete.get_microxs_and_flux(
    model=model,
    domains=[shield_cell],
    energies=[0, 30e6], # one energy bin from 0 to 30MeV
    chain_file=openmc.config['chain_file'],
)

constructing the operator, note we pass in the flux and micro xs calculated earlier

In [ ]:
operator = openmc.deplete.IndependentOperator(
    materials=materials,
    fluxes=[i[0] for i in flux_in_each_group],
    micros=micro_xs,
    reduce_chain=True,  # reduced to only the isotopes present in depletable materials and their possible progeny
    reduce_chain_level=5,
    normalization_mode="source-rate"
)

We define timesteps together with the source rate to make it clearer

In [ ]:
timesteps_and_source_rates = [
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),  # should saturate Ag110 here as it has been irradiated for over 5 halflives
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
]

# Uses list Python comprehension to get the timesteps and source_rates separately
timesteps = [item[0] for item in timesteps_and_source_rates]
source_rates = [item[1] for item in timesteps_and_source_rates]

construct the integrator

In [ ]:
integrator = openmc.deplete.PredictorIntegrator(
    operator=operator,
    timesteps=timesteps,
    source_rates=source_rates,
    timestep_units='s'
)

this runs the depltion calculations for the timesteps

In [ ]:
integrator.integrate()

Loads up the results


In [ ]:
results = openmc.deplete.Results.from_hdf5("depletion_results.h5")

Gets the material from the 2nd timestep and shows the composition

In [ ]:
second_time_step = results[2]
second_time_step.get_material('1')

prints the atoms of Ag110 in a table for reach time step

In [ ]:
times, number_of_Ag110_atoms = results.get_atoms(my_material, 'Ag110')
for time, num in zip(times, number_of_Ag110_atoms):
    print(f" Time {time}s. Number of Ag110 atoms {num}")

In [ ]:
# plots the number of atoms as a function of time

In [ ]:
plt.plot(times, number_of_Ag110_atoms)
plt.xlabel('Time [s]')
plt.ylabel('Number of Ag110 atoms')
plt.show()